In [1]:
import pandas as pd
import ast
import holidays

df = pd.read_csv("Data_from_script/output.csv", low_memory=False)

print(df)

print(df.shape)

        stop_id  route_id  run_id                  run_ref  direction_id  \
0         27234     11366      -1    59-15B--1-MF1-4307910           325   
1         27234     11366      -1    59-15B--1-MF1-4308510           325   
2         27234     11366      -1    59-15B--1-MF1-4309410           325   
3         27234     11366      -1    59-15B--1-MF1-4310310           325   
4         27234     11366      -1    59-15B--1-MF1-4311210           325   
...         ...       ...     ...                      ...           ...   
253168    18318     13545      -1    35-426--1-MF24-426300            30   
253169    18318     15065      -1  53-456--1-MF12-10602707            30   
253170    18318     13545      -1    35-426--1-MF24-426301            30   
253171    18318     13545      -1    35-426--1-MF24-426302            30   
253172    18318     13545      -1    35-426--1-MF24-426303            30   

       disruption_ids scheduled_departure_utc estimated_departure_utc  \
0            [

In [2]:
df["disruption_ids"] = df["disruption_ids"].apply(ast.literal_eval)

df["no_disruptions"] = df["disruption_ids"].apply(len)

In [3]:
#Use dt.date just for the date, similar to one in holidays.
df["estimated_departure_utc"] = pd.to_datetime(
    df["estimated_departure_utc"],
    errors="coerce")

df["scheduled_departure_utc"] = pd.to_datetime(df["scheduled_departure_utc"])


In [4]:
df.dropna(subset=['estimated_departure_utc'], inplace=True)

null_counts = df.isnull().sum()

print(null_counts)

stop_id                         0
route_id                        0
run_id                          0
run_ref                         0
direction_id                    0
disruption_ids                  0
scheduled_departure_utc         0
estimated_departure_utc         0
at_platform                     0
platform_number            140935
flags                      117618
departure_sequence              0
departure_note             140935
no_disruptions                  0
dtype: int64


In [5]:
list_of_holidays = []
vic_holidays = holidays.Australia(state='VIC', years=[2026])
for date in vic_holidays.keys():
    list_of_holidays.append(date)
print(list_of_holidays)

[datetime.date(2026, 4, 3), datetime.date(2026, 4, 6), datetime.date(2026, 1, 1), datetime.date(2026, 1, 26), datetime.date(2026, 3, 9), datetime.date(2026, 4, 4), datetime.date(2026, 4, 5), datetime.date(2026, 4, 25), datetime.date(2026, 6, 8), datetime.date(2026, 9, 25), datetime.date(2026, 11, 3), datetime.date(2026, 12, 25), datetime.date(2026, 12, 26), datetime.date(2026, 12, 28)]


In [6]:
#To do: Add a column that determines whether there is a public holiday on that day.

def public_holiday(row):
    return (row["scheduled_departure_utc"].date() in list_of_holidays)

df["Public Holidays"] = df.apply(public_holiday, axis = 1)

df["Public Holidays"] = df["Public Holidays"].astype(int)

In [7]:
print(df)

        stop_id  route_id  run_id                  run_ref  direction_id  \
12        27234     11366      -1    59-15B--1-MF1-4320910           325   
13        27234     11366      -1    59-15B--1-MF1-4322210           325   
14        27234     11366      -1    59-15B--1-MF1-4323210           325   
15        27234     11366      -1    59-15B--1-MF1-4324510           325   
16        27234     11366      -1    59-15B--1-MF1-4325510           325   
...         ...       ...     ...                      ...           ...   
253144    18318     13545      -1    35-426--1-MF24-426287            30   
253145    18318     15065      -1  53-456--1-MF12-10601968            30   
253146    18318     13545      -1    35-426--1-MF24-426288            30   
253147    18318     15065      -1   36-456--1-MF8-10601040            30   
253148    18318     13545      -1    35-426--1-MF24-426289            30   

       disruption_ids   scheduled_departure_utc   estimated_departure_utc  \
12        

In [8]:
df.drop('platform_number', axis=1, inplace=True)

In [9]:
df.drop('departure_note', axis=1, inplace=True)

In [10]:
df["flags"][df["flags"].notna()]

616       S_WCA
623       S_WCA
631       S_WCA
641       S_WCA
651       S_WCA
          ...  
252742    S_WCA
252743    S_WCA
252744    S_WCA
252746    S_WCA
252747    S_WCA
Name: flags, Length: 23317, dtype: object

In [11]:
df["flags"] = df["flags"].fillna(df["flags"].mode().iloc[0])

In [12]:
null_counts = df.isnull().sum()

print(null_counts)

stop_id                    0
route_id                   0
run_id                     0
run_ref                    0
direction_id               0
disruption_ids             0
scheduled_departure_utc    0
estimated_departure_utc    0
at_platform                0
flags                      0
departure_sequence         0
no_disruptions             0
Public Holidays            0
dtype: int64


In [13]:
print(df.shape)

(140935, 13)


In [14]:
df["delay"] = df["estimated_departure_utc"] - df["scheduled_departure_utc"]

In [15]:
print(df)

        stop_id  route_id  run_id                  run_ref  direction_id  \
12        27234     11366      -1    59-15B--1-MF1-4320910           325   
13        27234     11366      -1    59-15B--1-MF1-4322210           325   
14        27234     11366      -1    59-15B--1-MF1-4323210           325   
15        27234     11366      -1    59-15B--1-MF1-4324510           325   
16        27234     11366      -1    59-15B--1-MF1-4325510           325   
...         ...       ...     ...                      ...           ...   
253144    18318     13545      -1    35-426--1-MF24-426287            30   
253145    18318     15065      -1  53-456--1-MF12-10601968            30   
253146    18318     13545      -1    35-426--1-MF24-426288            30   
253147    18318     15065      -1   36-456--1-MF8-10601040            30   
253148    18318     13545      -1    35-426--1-MF24-426289            30   

       disruption_ids   scheduled_departure_utc   estimated_departure_utc  \
12        

In [16]:
df["Public Holidays"] = df["Public Holidays"].astype(bool)

In [17]:
print(df.dtypes)

stop_id                                  int64
route_id                                 int64
run_id                                   int64
run_ref                                 object
direction_id                             int64
disruption_ids                          object
scheduled_departure_utc    datetime64[ns, UTC]
estimated_departure_utc    datetime64[ns, UTC]
at_platform                               bool
flags                                   object
departure_sequence                       int64
no_disruptions                           int64
Public Holidays                           bool
delay                          timedelta64[ns]
dtype: object


In [18]:
df["scheduled_hour"] = df["scheduled_departure_utc"].dt.hour

In [19]:
df["scheduled_day"] = df["scheduled_departure_utc"].dt.day_name()

In [20]:
print(df.dtypes)

stop_id                                  int64
route_id                                 int64
run_id                                   int64
run_ref                                 object
direction_id                             int64
disruption_ids                          object
scheduled_departure_utc    datetime64[ns, UTC]
estimated_departure_utc    datetime64[ns, UTC]
at_platform                               bool
flags                                   object
departure_sequence                       int64
no_disruptions                           int64
Public Holidays                           bool
delay                          timedelta64[ns]
scheduled_hour                           int32
scheduled_day                           object
dtype: object


In [21]:
df["delay_in_seconds"] = df["delay"].dt.total_seconds()

In [22]:
print(df.head())

    stop_id  route_id  run_id                run_ref  direction_id  \
12    27234     11366      -1  59-15B--1-MF1-4320910           325   
13    27234     11366      -1  59-15B--1-MF1-4322210           325   
14    27234     11366      -1  59-15B--1-MF1-4323210           325   
15    27234     11366      -1  59-15B--1-MF1-4324510           325   
16    27234     11366      -1  59-15B--1-MF1-4325510           325   

   disruption_ids   scheduled_departure_utc   estimated_departure_utc  \
12       [327040] 2025-12-12 00:40:00+00:00 2025-12-12 00:42:40+00:00   
13       [327040] 2025-12-12 01:10:00+00:00 2025-12-12 01:11:01+00:00   
14       [327040] 2025-12-12 01:40:00+00:00 2025-12-12 01:44:51+00:00   
15       [327040] 2025-12-12 02:10:00+00:00 2025-12-12 02:15:23+00:00   
16       [327040] 2025-12-12 02:40:00+00:00 2025-12-12 02:40:34+00:00   

    at_platform  flags  departure_sequence  no_disruptions  Public Holidays  \
12         True  S_WCA                   0               1   

In [23]:
from meteostat import Stations
stations = Stations()
stations = stations.nearby(-37.8142, 144.9632)
station = stations.fetch(1)

#print(station)

#Use id 95873

In [24]:
#df.to_csv("test.csv", index = False)